In [1]:
import numpy as np

In [2]:
def evaluation_forward(A, B, PI, O, alpha):
    alpha[:, 0] = np.multiply(PI, B[:, O[0]])
    for t in range(1, O.shape[0]):
        for j in range(A.shape[0]):
            alpha[j, t] = np.sum(np.multiply(alpha[:, t - 1], A[:, j])) * B[j, O[t]]
    return alpha

In [3]:
def evaluation_backward(A, B, PI, O, beta):
    beta[:, -1] = np.ones((A.shape[0]), dtype=np.float64)
    for t in range(O.shape[0] - 2, -1, -1):
        for i in range(A.shape[0]):
            beta[i, t] = np.sum(np.multiply(np.multiply(A[i, :], B[:, O[t]]), beta[:, t + 1]))
    return beta

In [4]:
def decode(A, B, PI, O, delta):
    delta[:, 0] = np.multiply(PI, B[:, O[0]])
    for t in range(1, O.shape[0]):
        for j in range(A.shape[0]):
            delta[j, t] = np.amax(np.multiply(delta[:, t - 1], A[:, j])) * B[j, O[t]]
    path = np.argmax(delta, axis=0) + 1
    return delta, path

In [5]:
def learn(A, B, PI, O, V, alpha, beta, XI, gamma, epsilon=10e-12):
    alpha = evaluation_forward(A, B, PI, O, alpha)
    beta = evaluation_backward(A, B, PI, O, beta)
    
    #Compute XI#
    p = np.sum(np.multiply(alpha, beta), axis=0)
    for t in range(O.shape[0] - 1):
        for i in range(A.shape[0]):
            for j in range(A.shape[0]):
                XI[i, j, t] = (alpha[i, t] * beta[j, t + 1] * A[i, j] * B[j, O[t + 1]]) / (p[t] + epsilon)
    
    #Compute gamma#
    for t in range(O.shape[0]):
        for i in range(A.shape[0]):
            gamma[i, t] = np.sum(XI[i, :, t])
            
    #Compute new A#
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            A[i, j] = np.sum(XI[i, j, :]) / (np.sum(gamma[i]) + epsilon)

    #Compute new PI#
    for i in range(A.shape[0]):
        PI[i] = gamma[i, 0]
        
    #Compute new B#
    for j in range(A.shape[0]):
        for k in range(V.shape[0]):
            B[j, k] = np.sum(np.multiply(gamma[j], (O == V[k]).astype(np.float64))) / (np.sum(gamma[j]) + epsilon)
    return A, B, PI, gamma, XI, alpha, beta

In [6]:
A = np.array([ #1st Rank = From State, 2nd Rank = To State
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3]
], dtype=np.float64)
PI = np.array([1/ 3, 1 / 3, 1 / 3], dtype=np.float64) #1st Rank = Init State
B = np.array([ #1st Rankg = State, 2nd Rank = Output Value
    [4 / 6, 2 / 6],
    [2 / 6, 4 / 6],
    [3 / 6, 3 / 6]
], dtype=np.float64)
O = np.array([0, 0, 1, 0, 1], dtype=np.int32) #0 = red, 1 = blue
V = np.array([0, 1], dtype=np.int32) #0 = red, 1 = blue
alpha = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
beta = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
delta = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
gamma = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
XI = np.zeros((B.shape[0], B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = From State, 2nd Rank = To State, 3rd Rank = Time

In [7]:
records = list()
records.append((np.copy(A), np.copy(B), np.copy(PI)))
for i in range(10000):
    A, B, PI, gamma, XI, alpha, beta = learn(A, B, PI, O, V, alpha, beta, XI, gamma)
    delta, path = decode(A, B, PI, O, delta)
    records.append((np.copy(A), np.copy(B), np.copy(PI)))
    print('Iters {}, Path: {}'.format(i + 1, path))

Iters 1, Path: [1 1 2 1 2]
Iters 2, Path: [1 1 2 1 2]
Iters 3, Path: [1 1 2 1 2]
Iters 4, Path: [1 1 2 1 2]
Iters 5, Path: [1 3 2 1 2]
Iters 6, Path: [1 1 2 1 2]
Iters 7, Path: [1 3 2 1 2]
Iters 8, Path: [1 1 2 1 2]
Iters 9, Path: [1 3 3 1 2]
Iters 10, Path: [1 1 3 1 3]
Iters 11, Path: [1 3 3 3 3]
Iters 12, Path: [1 3 3 1 3]
Iters 13, Path: [1 3 3 3 3]
Iters 14, Path: [1 3 3 1 3]
Iters 15, Path: [1 3 3 3 3]
Iters 16, Path: [1 1 3 1 3]
Iters 17, Path: [1 3 3 3 3]
Iters 18, Path: [1 1 3 1 3]
Iters 19, Path: [1 3 3 3 3]
Iters 20, Path: [1 1 3 1 3]
Iters 21, Path: [1 3 3 3 3]
Iters 22, Path: [1 1 3 1 3]
Iters 23, Path: [1 3 3 3 3]
Iters 24, Path: [1 1 3 1 3]
Iters 25, Path: [1 3 3 3 3]
Iters 26, Path: [1 1 3 1 3]
Iters 27, Path: [1 3 3 3 3]
Iters 28, Path: [1 1 3 1 3]
Iters 29, Path: [1 3 3 3 3]
Iters 30, Path: [1 1 3 1 3]
Iters 31, Path: [1 3 3 3 3]
Iters 32, Path: [1 1 3 1 3]
Iters 33, Path: [1 3 3 3 3]
Iters 34, Path: [1 1 3 1 3]
Iters 35, Path: [1 3 3 3 3]
Iters 36, Path: [1 1 3 1 3]
I

Iters 493, Path: [1 3 3 3 3]
Iters 494, Path: [1 3 3 3 3]
Iters 495, Path: [1 3 3 3 3]
Iters 496, Path: [1 1 3 1 3]
Iters 497, Path: [1 3 3 3 3]
Iters 498, Path: [1 3 3 3 3]
Iters 499, Path: [1 3 3 3 3]
Iters 500, Path: [1 1 3 1 3]
Iters 501, Path: [1 3 3 3 3]
Iters 502, Path: [1 3 3 3 3]
Iters 503, Path: [1 3 3 3 3]
Iters 504, Path: [1 1 3 1 3]
Iters 505, Path: [1 3 3 3 3]
Iters 506, Path: [1 3 3 3 3]
Iters 507, Path: [1 3 3 3 3]
Iters 508, Path: [1 1 3 1 3]
Iters 509, Path: [1 3 3 3 3]
Iters 510, Path: [1 3 3 3 3]
Iters 511, Path: [1 3 3 3 3]
Iters 512, Path: [1 1 3 1 3]
Iters 513, Path: [1 3 3 3 3]
Iters 514, Path: [1 3 3 3 3]
Iters 515, Path: [1 3 3 3 3]
Iters 516, Path: [1 1 3 1 3]
Iters 517, Path: [1 3 3 3 3]
Iters 518, Path: [1 3 3 3 3]
Iters 519, Path: [1 3 3 3 3]
Iters 520, Path: [1 1 3 1 3]
Iters 521, Path: [1 3 3 3 3]
Iters 522, Path: [1 3 3 3 3]
Iters 523, Path: [1 3 3 3 3]
Iters 524, Path: [1 1 3 1 3]
Iters 525, Path: [1 3 3 3 3]
Iters 526, Path: [1 3 3 3 3]
Iters 527, Pat

Iters 1507, Path: [1 3 3 3 3]
Iters 1508, Path: [1 1 3 1 3]
Iters 1509, Path: [1 3 3 3 3]
Iters 1510, Path: [1 3 3 3 3]
Iters 1511, Path: [1 3 3 3 3]
Iters 1512, Path: [1 1 3 1 3]
Iters 1513, Path: [1 3 3 3 3]
Iters 1514, Path: [1 3 3 3 3]
Iters 1515, Path: [1 3 3 3 3]
Iters 1516, Path: [1 1 3 1 3]
Iters 1517, Path: [1 3 3 3 3]
Iters 1518, Path: [1 3 3 3 3]
Iters 1519, Path: [1 3 3 3 3]
Iters 1520, Path: [1 1 3 1 3]
Iters 1521, Path: [1 3 3 3 3]
Iters 1522, Path: [1 3 3 3 3]
Iters 1523, Path: [1 3 3 3 3]
Iters 1524, Path: [1 1 3 1 3]
Iters 1525, Path: [1 3 3 3 3]
Iters 1526, Path: [1 3 3 3 3]
Iters 1527, Path: [1 3 3 3 3]
Iters 1528, Path: [1 1 3 1 3]
Iters 1529, Path: [1 3 3 3 3]
Iters 1530, Path: [1 3 3 3 3]
Iters 1531, Path: [1 3 3 3 3]
Iters 1532, Path: [1 1 3 1 3]
Iters 1533, Path: [1 3 3 3 3]
Iters 1534, Path: [1 3 3 3 3]
Iters 1535, Path: [1 3 3 3 3]
Iters 1536, Path: [1 1 3 1 3]
Iters 1537, Path: [1 3 3 3 3]
Iters 1538, Path: [1 3 3 3 3]
Iters 1539, Path: [1 3 3 3 3]
Iters 1540

Iters 2546, Path: [1 3 3 3 3]
Iters 2547, Path: [1 3 3 3 3]
Iters 2548, Path: [1 1 3 1 3]
Iters 2549, Path: [1 3 3 3 3]
Iters 2550, Path: [1 3 3 3 3]
Iters 2551, Path: [1 3 3 3 3]
Iters 2552, Path: [1 1 3 1 3]
Iters 2553, Path: [1 3 3 3 3]
Iters 2554, Path: [1 3 3 3 3]
Iters 2555, Path: [1 3 3 3 3]
Iters 2556, Path: [1 1 3 1 3]
Iters 2557, Path: [1 3 3 3 3]
Iters 2558, Path: [1 3 3 3 3]
Iters 2559, Path: [1 3 3 3 3]
Iters 2560, Path: [1 1 3 1 3]
Iters 2561, Path: [1 3 3 3 3]
Iters 2562, Path: [1 3 3 3 3]
Iters 2563, Path: [1 3 3 3 3]
Iters 2564, Path: [1 1 3 1 3]
Iters 2565, Path: [1 3 3 3 3]
Iters 2566, Path: [1 3 3 3 3]
Iters 2567, Path: [1 3 3 3 3]
Iters 2568, Path: [1 1 3 1 3]
Iters 2569, Path: [1 3 3 3 3]
Iters 2570, Path: [1 3 3 3 3]
Iters 2571, Path: [1 3 3 3 3]
Iters 2572, Path: [1 1 3 1 3]
Iters 2573, Path: [1 3 3 3 3]
Iters 2574, Path: [1 3 3 3 3]
Iters 2575, Path: [1 3 3 3 3]
Iters 2576, Path: [1 1 3 1 3]
Iters 2577, Path: [1 3 3 3 3]
Iters 2578, Path: [1 3 3 3 3]
Iters 2579

Iters 3535, Path: [1 3 3 3 3]
Iters 3536, Path: [1 1 3 1 3]
Iters 3537, Path: [1 3 3 3 3]
Iters 3538, Path: [1 3 3 3 3]
Iters 3539, Path: [1 3 3 3 3]
Iters 3540, Path: [1 1 3 1 3]
Iters 3541, Path: [1 3 3 3 3]
Iters 3542, Path: [1 3 3 3 3]
Iters 3543, Path: [1 3 3 3 3]
Iters 3544, Path: [1 1 3 1 3]
Iters 3545, Path: [1 3 3 3 3]
Iters 3546, Path: [1 3 3 3 3]
Iters 3547, Path: [1 3 3 3 3]
Iters 3548, Path: [1 1 3 1 3]
Iters 3549, Path: [1 3 3 3 3]
Iters 3550, Path: [1 3 3 3 3]
Iters 3551, Path: [1 3 3 3 3]
Iters 3552, Path: [1 1 3 1 3]
Iters 3553, Path: [1 3 3 3 3]
Iters 3554, Path: [1 3 3 3 3]
Iters 3555, Path: [1 3 3 3 3]
Iters 3556, Path: [1 1 3 1 3]
Iters 3557, Path: [1 3 3 3 3]
Iters 3558, Path: [1 3 3 3 3]
Iters 3559, Path: [1 3 3 3 3]
Iters 3560, Path: [1 1 3 1 3]
Iters 3561, Path: [1 3 3 3 3]
Iters 3562, Path: [1 3 3 3 3]
Iters 3563, Path: [1 3 3 3 3]
Iters 3564, Path: [1 1 3 1 3]
Iters 3565, Path: [1 3 3 3 3]
Iters 3566, Path: [1 3 3 3 3]
Iters 3567, Path: [1 3 3 3 3]
Iters 3568

Iters 4567, Path: [1 3 3 3 3]
Iters 4568, Path: [1 1 3 1 3]
Iters 4569, Path: [1 3 3 3 3]
Iters 4570, Path: [1 3 3 3 3]
Iters 4571, Path: [1 3 3 3 3]
Iters 4572, Path: [1 1 3 1 3]
Iters 4573, Path: [1 3 3 3 3]
Iters 4574, Path: [1 3 3 3 3]
Iters 4575, Path: [1 3 3 3 3]
Iters 4576, Path: [1 1 3 1 3]
Iters 4577, Path: [1 3 3 3 3]
Iters 4578, Path: [1 3 3 3 3]
Iters 4579, Path: [1 3 3 3 3]
Iters 4580, Path: [1 1 3 1 3]
Iters 4581, Path: [1 3 3 3 3]
Iters 4582, Path: [1 3 3 3 3]
Iters 4583, Path: [1 3 3 3 3]
Iters 4584, Path: [1 1 3 1 3]
Iters 4585, Path: [1 3 3 3 3]
Iters 4586, Path: [1 3 3 3 3]
Iters 4587, Path: [1 3 3 3 3]
Iters 4588, Path: [1 1 3 1 3]
Iters 4589, Path: [1 3 3 3 3]
Iters 4590, Path: [1 3 3 3 3]
Iters 4591, Path: [1 3 3 3 3]
Iters 4592, Path: [1 1 3 1 3]
Iters 4593, Path: [1 3 3 3 3]
Iters 4594, Path: [1 3 3 3 3]
Iters 4595, Path: [1 3 3 3 3]
Iters 4596, Path: [1 1 3 1 3]
Iters 4597, Path: [1 3 3 3 3]
Iters 4598, Path: [1 3 3 3 3]
Iters 4599, Path: [1 3 3 3 3]
Iters 4600

Iters 5557, Path: [1 3 3 3 3]
Iters 5558, Path: [1 3 3 3 3]
Iters 5559, Path: [1 3 3 3 3]
Iters 5560, Path: [1 1 3 1 3]
Iters 5561, Path: [1 3 3 3 3]
Iters 5562, Path: [1 3 3 3 3]
Iters 5563, Path: [1 3 3 3 3]
Iters 5564, Path: [1 1 3 1 3]
Iters 5565, Path: [1 3 3 3 3]
Iters 5566, Path: [1 3 3 3 3]
Iters 5567, Path: [1 3 3 3 3]
Iters 5568, Path: [1 1 3 1 3]
Iters 5569, Path: [1 3 3 3 3]
Iters 5570, Path: [1 3 3 3 3]
Iters 5571, Path: [1 3 3 3 3]
Iters 5572, Path: [1 1 3 1 3]
Iters 5573, Path: [1 3 3 3 3]
Iters 5574, Path: [1 3 3 3 3]
Iters 5575, Path: [1 3 3 3 3]
Iters 5576, Path: [1 1 3 1 3]
Iters 5577, Path: [1 3 3 3 3]
Iters 5578, Path: [1 3 3 3 3]
Iters 5579, Path: [1 3 3 3 3]
Iters 5580, Path: [1 1 3 1 3]
Iters 5581, Path: [1 3 3 3 3]
Iters 5582, Path: [1 3 3 3 3]
Iters 5583, Path: [1 3 3 3 3]
Iters 5584, Path: [1 1 3 1 3]
Iters 5585, Path: [1 3 3 3 3]
Iters 5586, Path: [1 3 3 3 3]
Iters 5587, Path: [1 3 3 3 3]
Iters 5588, Path: [1 1 3 1 3]
Iters 5589, Path: [1 3 3 3 3]
Iters 5590

Iters 6552, Path: [1 1 3 1 3]
Iters 6553, Path: [1 3 3 3 3]
Iters 6554, Path: [1 3 3 3 3]
Iters 6555, Path: [1 3 3 3 3]
Iters 6556, Path: [1 1 3 1 3]
Iters 6557, Path: [1 3 3 3 3]
Iters 6558, Path: [1 3 3 3 3]
Iters 6559, Path: [1 3 3 3 3]
Iters 6560, Path: [1 1 3 1 3]
Iters 6561, Path: [1 3 3 3 3]
Iters 6562, Path: [1 3 3 3 3]
Iters 6563, Path: [1 3 3 3 3]
Iters 6564, Path: [1 1 3 1 3]
Iters 6565, Path: [1 3 3 3 3]
Iters 6566, Path: [1 3 3 3 3]
Iters 6567, Path: [1 3 3 3 3]
Iters 6568, Path: [1 1 3 1 3]
Iters 6569, Path: [1 3 3 3 3]
Iters 6570, Path: [1 3 3 3 3]
Iters 6571, Path: [1 3 3 3 3]
Iters 6572, Path: [1 1 3 1 3]
Iters 6573, Path: [1 3 3 3 3]
Iters 6574, Path: [1 3 3 3 3]
Iters 6575, Path: [1 3 3 3 3]
Iters 6576, Path: [1 1 3 1 3]
Iters 6577, Path: [1 3 3 3 3]
Iters 6578, Path: [1 3 3 3 3]
Iters 6579, Path: [1 3 3 3 3]
Iters 6580, Path: [1 1 3 1 3]
Iters 6581, Path: [1 3 3 3 3]
Iters 6582, Path: [1 3 3 3 3]
Iters 6583, Path: [1 3 3 3 3]
Iters 6584, Path: [1 1 3 1 3]
Iters 6585

Iters 7546, Path: [1 3 3 3 3]
Iters 7547, Path: [1 3 3 3 3]
Iters 7548, Path: [1 1 3 1 3]
Iters 7549, Path: [1 3 3 3 3]
Iters 7550, Path: [1 3 3 3 3]
Iters 7551, Path: [1 3 3 3 3]
Iters 7552, Path: [1 1 3 1 3]
Iters 7553, Path: [1 3 3 3 3]
Iters 7554, Path: [1 3 3 3 3]
Iters 7555, Path: [1 3 3 3 3]
Iters 7556, Path: [1 1 3 1 3]
Iters 7557, Path: [1 3 3 3 3]
Iters 7558, Path: [1 3 3 3 3]
Iters 7559, Path: [1 3 3 3 3]
Iters 7560, Path: [1 1 3 1 3]
Iters 7561, Path: [1 3 3 3 3]
Iters 7562, Path: [1 3 3 3 3]
Iters 7563, Path: [1 3 3 3 3]
Iters 7564, Path: [1 1 3 1 3]
Iters 7565, Path: [1 3 3 3 3]
Iters 7566, Path: [1 3 3 3 3]
Iters 7567, Path: [1 3 3 3 3]
Iters 7568, Path: [1 1 3 1 3]
Iters 7569, Path: [1 3 3 3 3]
Iters 7570, Path: [1 3 3 3 3]
Iters 7571, Path: [1 3 3 3 3]
Iters 7572, Path: [1 1 3 1 3]
Iters 7573, Path: [1 3 3 3 3]
Iters 7574, Path: [1 3 3 3 3]
Iters 7575, Path: [1 3 3 3 3]
Iters 7576, Path: [1 1 3 1 3]
Iters 7577, Path: [1 3 3 3 3]
Iters 7578, Path: [1 3 3 3 3]
Iters 7579

Iters 8551, Path: [1 3 3 3 3]
Iters 8552, Path: [1 1 3 1 3]
Iters 8553, Path: [1 3 3 3 3]
Iters 8554, Path: [1 3 3 3 3]
Iters 8555, Path: [1 3 3 3 3]
Iters 8556, Path: [1 1 3 1 3]
Iters 8557, Path: [1 3 3 3 3]
Iters 8558, Path: [1 3 3 3 3]
Iters 8559, Path: [1 3 3 3 3]
Iters 8560, Path: [1 1 3 1 3]
Iters 8561, Path: [1 3 3 3 3]
Iters 8562, Path: [1 3 3 3 3]
Iters 8563, Path: [1 3 3 3 3]
Iters 8564, Path: [1 1 3 1 3]
Iters 8565, Path: [1 3 3 3 3]
Iters 8566, Path: [1 3 3 3 3]
Iters 8567, Path: [1 3 3 3 3]
Iters 8568, Path: [1 1 3 1 3]
Iters 8569, Path: [1 3 3 3 3]
Iters 8570, Path: [1 3 3 3 3]
Iters 8571, Path: [1 3 3 3 3]
Iters 8572, Path: [1 1 3 1 3]
Iters 8573, Path: [1 3 3 3 3]
Iters 8574, Path: [1 3 3 3 3]
Iters 8575, Path: [1 3 3 3 3]
Iters 8576, Path: [1 1 3 1 3]
Iters 8577, Path: [1 3 3 3 3]
Iters 8578, Path: [1 3 3 3 3]
Iters 8579, Path: [1 3 3 3 3]
Iters 8580, Path: [1 1 3 1 3]
Iters 8581, Path: [1 3 3 3 3]
Iters 8582, Path: [1 3 3 3 3]
Iters 8583, Path: [1 3 3 3 3]
Iters 8584

In [8]:
for i in range(9997, 10001):
    print('============ Iter {} ============'.format(i))
    print('A:\n{}'.format(records[i][0]))
    print('B:\n{}'.format(records[i][1]))
    print('PI:\n{}'.format(records[i][2]))

============ Iter 9997 ============
A:
[[1.11270653e-07 0.00000000e+00 9.99999888e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.33546929e-03 0.00000000e+00 9.98657059e-01]]
B:
[[1.         0.        ]
 [0.         0.        ]
 [0.99865692 0.00133561]]
PI:
[1.80200478e-09 0.00000000e+00 0.00000000e+00]
============ Iter 9998 ============
A:
[[1.11420532e-07 0.00000000e+00 9.99999889e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.33440549e-03 0.00000000e+00 9.98665594e-01]]
B:
[[1.        0.       ]
 [0.        0.       ]
 [0.0021499 0.9978501]]
PI:
[0.1931477 0.        0.       ]
============ Iter 9999 ============
A:
[[2.46635244e-07 0.00000000e+00 9.99999753e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.68121489e-06 0.00000000e+00 9.99998319e-01]]
B:
[[1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [9.99993939e-01 6.06139305e-06]]
PI:
[0.99801121 0.         0.        ]
============ Iter 10000 ============
A:
[[2.46636739e-07 0.00000000